In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib
import geopandas as gp
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
import Outils

In [2]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [ ]:
toto=at.affecter_troncon_ligne('TRONROUT0000000109674477')

In [ ]:
toto

In [3]:
dico=at.affecter_troncon(df)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000113489744 à 15:33:32 nb ligne traite : 0, nb ligne differente=0


TypeError: recup_troncon_parallele() takes 1 positional argument but 2 were given

In [ ]:
a